Set up GRASS environment

In [ ]:
import os
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [ ]:
!g.extension r.sample.category
# r.object.geometry is r.neighborhoodmatrix
!g.extension r.neighborhoodmatrix

Import all data and set region to NC extent* and 30m resolution  
*data are imported at the NC extent in preparation for future simulations to be computed at the NC extent, for the purposes of this assignment I will be running simulations on a restricted area (see cell 6)

In [ ]:
# import data
!v.import input=D:\Zoning_Development\input_data\NCDOT_County_Boundaries.shp output=counties
# set region to NC counties and resolution to 30m 
!g.region -a vector=counties@input res=30
# roads
!v.import output=roads extent=region input=D:\Zoning_Development\input_data\tl_2021_37_prisecroads.shp layer=tl_2021_37_prisecroads
# rails
!v.import output=rails extent=region input=D:\Zoning_Development\input_data\tl_2021_us_rails.shp layer=tl_2021_us_rails
# protected areas
!v.import output=protected extent=region input=D:\Zoning_Development\input_data\PADUS3_0Combined_StateNC.shp layer=PADUS3_0Combined_StateNC
# slope
!r.import extent=region resolution=region input=D:\Zoning\DEM\slope.tif output=slope
# city boundaries
!v.import output=city_boundaries extent=region input=D:\Zoning_Development\input_data\NCDOT_City_Boundaries.shp layer=NCDOT_City_Boundaries
# intersections
!v.import output=interchanges extent=region input=D:\Zoning\interchanges\NC_interchange.shp layer=NC_interchange
# import all years of NLCD
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2021_land_cover_l48_20230630.img output=nlcd_2021
!r.import extent=region resolution=region input=D:\Zoning\nlcd2019\nlcd.tif output=nlcd_2019
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2016_land_cover_l48_20210604.img output=nlcd_2016
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2013_land_cover_l48_20210604.img output=nlcd_2013
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2011_land_cover_l48_20210604.img output=nlcd_2011
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2008_land_cover_l48_20210604.img output=nlcd_2008
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2006_land_cover_l48_20210604.img output=nlcd_2006
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2004_land_cover_l48_20210604.img output=nlcd_2004
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\nlcd_2001_land_cover_l48_20210604.img output=nlcd_2001
# import zoning data
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\zoning_core_reclass.tif output=zoning_core
!r.import extent=region resolution=region input=D:\Zoning_Development\input_data\zoning_sub_reclass.tif output=zoning_sub

Restrict extent to the counties of the Central Pines Regional Council (7 counties in the Triangle region in central NC)

In [ ]:
# limiting extent to Central Pines Regional Council
!r.mask vector=central_pines_regional_council@input
# mask doesn't effect vectors so need to clip any vector data

Reclassify land cover data as developed or undeveloped

In [ ]:
gs.mapcalc("urban_2001 = if(nlcd_2001 >= 21 && nlcd_2001 <= 24, 1, if(nlcd_2001 == 11 || nlcd_2001 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2004 = if(nlcd_2004 >= 21 && nlcd_2004 <= 24, 1, if(nlcd_2004 == 11 || nlcd_2004 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2006 = if(nlcd_2006 >= 21 && nlcd_2006 <= 24, 1, if(nlcd_2006 == 11 || nlcd_2006 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2008 = if(nlcd_2008 >= 21 && nlcd_2008 <= 24, 1, if(nlcd_2008 == 11 || nlcd_2008 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2011 = if(nlcd_2011 >= 21 && nlcd_2011 <= 24, 1, if(nlcd_2011 == 11 || nlcd_2011 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2013 = if(nlcd_2013 >= 21 && nlcd_2013 <= 24, 1, if(nlcd_2013 == 11 || nlcd_2013 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2016 = if(nlcd_2016 >= 20 && nlcd_2016 <= 24, 1, if(nlcd_2016 == 11 || nlcd_2016 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2019 = if(nlcd_2019 >= 20 && nlcd_2019 <= 24, 1, if(nlcd_2019 == 11 || nlcd_2019 >= 90 || protected, null(), 0))")
gs.mapcalc("urban_2021 = if(nlcd_2021 >= 20 && nlcd_2021 <= 24, 1, if(nlcd_2021 == 11 || nlcd_2021 >= 90 || protected, null(), 0))")

Convert relevant vector data to raster and calculate distance rasters

In [ ]:
!v.to.rast input=protected output=protected use=val
# !r.null map=protected null=0
# show('protected')
!r.null map=protected setnull=0
!r.grow.distance input=protected distance=dist_to_protected
!r.colors map=dist_to_protected color=gyr -e
show('dist_to_protected')

In [ ]:
!v.to.rast input=central_pines_regional_council type=area use=attr attribute_column=FIPS output=sa_counties
gs.mapcalc("water = if(nlcd_2021 == 11, 1, null())")
!r.grow.distance input=water distance=dist_to_water
!r.colors map=dist_to_water color=blues -en
show('dist_to_water')

In [ ]:
gs.mapcalc("forest = if(nlcd_2021 >= 41 && nlcd_2021 <= 43, 1, 0)")
gs.mapcalc("forest_2001 = if(nlcd_2001 >= 40 && nlcd_2001 <= 43, 1, 0)")
!r.neighbors input=forest output=forest_smooth size=37 method=average -c
!r.neighbors input=forest_2001 output=forest_2001_smooth size=37 method=average -c
!r.colors map=forest_smooth,forest_2001_smooth color=ndvi
show('forest_smooth')

In [ ]:
!v.to.rast input=city_boundaries type=area use=val output=city_boundaries
!r.grow.distance input=city_boundaries distance=dist_to_city_boundaries
!v.to.rast input=interchanges type=point use=val output=interchanges
!r.grow.distance input=interchanges distance=dist_to_interchanges
show('dist_to_interchanges')

In [ ]:
!v.to.rast input=roads type=line use=val output=roads
!r.null map=roads null=0
!r.neighbors input=roads output=road_dens size=37 method=average -c --overwrite
show('road_dens')

Calculating development pressure based on urban development in 2001 and 2011

In [ ]:
gs.mapcalc('urban_2001_nonull = if(isnull(urban_2001), 0, urban_2001)')
gs.mapcalc('urban_2011_nonull = if(isnull(urban_2011), 0, urban_2011)')
#TODO check these values
!r.futures.devpressure -n input=urban_2001_nonull output=devpressure_30_05_01_2001 method=gravity size=37 gamma=0.5 scaling_factor=0.1
!r.futures.devpressure -n input=urban_2011_nonull output=devpressure_30_05_01_2011 method=gravity size=37 gamma=0.5 scaling_factor=0.1
show('devpressure_30_05_01_2011')

Seperate out specific zoning districts and calculate distance rasters

In [ ]:
# Will first just try distance to each core zoning districts
# distance to residential
gs.mapcalc("residential = if(zoning_core == 1, 1, null())")
!r.grow.distance input=residential distance=dist_to_residential
# distance to non-res
gs.mapcalc("nonresidential = if(zoning_core == 2, 1, null())")
!r.grow.distance input=nonresidential distance=dist_to_nonresidential
# distance to mixed-use
gs.mapcalc("mixeduse = if(zoning_core == 3, 1, null())")
!r.grow.distance input=mixeduse distance=dist_to_mixeduse

# Then try distance to specific sub-district zones
# distance to medium-low, medium, or high-density residential
gs.mapcalc("high_den_residential = if(zoning_sub <= 120, 1, null())")
!r.grow.distance input=high_den_residential distance=dist_to_high_den_residential
# distance to low-density and agricultural residential
gs.mapcalc("low_den_residential = if(zoning_sub > 120 && zoning_sub < 200, 1, null())")
!r.grow.distance input=low_den_residential distance=dist_to_low_den_residential
# distance to industrial
gs.mapcalc("industrial = if(zoning_sub == 201, 1, null())")
!r.grow.distance input=industrial distance=dist_to_industrial
# distance to commercial
gs.mapcalc("commercial = if(zoning_sub == 200, 1, null())")
!r.grow.distance input=commercial distance=dist_to_commercial
# distance to planned development
gs.mapcalc("planned_dev = if(zoning_sub == 301, 1, null())")
!r.grow.distance input=planned_dev distance=dist_to_planned_dev
# distance to downtown
gs.mapcalc("downtown = if(zoning_sub == 302, 1, null())")
!r.grow.distance input=downtown distance=dist_to_downtown

Check range of values in all predictors to determine if rescaling is necessary

In [ ]:
for name in ['slope', 'dist_to_water', 'dist_to_protected',
             'forest_smooth', 'forest_2001_smooth', 'dist_to_city_boundaries', 'dist_to_interchanges',
             'road_dens', 'devpressure_30_05_01_2001', 'devpressure_30_05_01_2021',
             'dist_to_downtown', 'dist_to_planned_dev', 'dist_to_commercial', 'dist_to_industrial', 
             'dist_to_low_den_residential', 'dist_to_high_den_residential', 'dist_to_mixeduse',
             'dist_to_nonresidential', 'dist_to_residential']:
    minmax = gs.raster_info(name)
    print (name, minmax['min'], minmax['max'])

rescale based on results

In [ ]:
gs.mapcalc("dist_to_water_km = dist_to_water / 1000")
gs.mapcalc("dist_to_protected_km = dist_to_protected / 1000")
gs.mapcalc("dist_to_city_boundaries_km = dist_to_city_boundaries / 1000")
gs.mapcalc("dist_to_interchanges_km = dist_to_interchanges / 1000")
gs.mapcalc("road_dens_perc = road_dens * 100")
gs.mapcalc("forest_smooth_perc = forest_smooth * 100")
gs.mapcalc("forest_2001_smooth_perc = forest_smooth * 100")
gs.mapcalc("dist_to_downtown_km = dist_to_downtown / 1000")
gs.mapcalc("dist_to_planned_dev_km = dist_to_planned_dev / 1000")
gs.mapcalc("dist_to_commercial_km = dist_to_commercial / 1000")
gs.mapcalc("dist_to_industrial_km = dist_to_industrial / 1000")
gs.mapcalc("dist_to_low_den_residential_km = dist_to_low_den_residential / 1000")
gs.mapcalc("dist_to_high_den_residential_km = dist_to_high_den_residential / 1000")
gs.mapcalc("dist_to_mixeduse_km = dist_to_mixeduse / 1000")
gs.mapcalc("dist_to_nonresidential_km = dist_to_nonresidential / 1000")
gs.mapcalc("dist_to_residential_km = dist_to_residential / 1000")

Double check if rescaling was successful

In [ ]:
for name in ['dist_to_water_km','dist_to_protected_km','forest_smooth_perc',
             'forest_2001_smooth_perc','dist_to_city_boundaries_km','dist_to_interchanges_km','road_dens_perc',
             'dist_to_downtown_km','dist_to_planned_dev_km','dist_to_commercial_km','dist_to_industrial_km',
             'dist_to_low_den_residential_km','dist_to_high_den_residential_km','dist_to_mixeduse_km',
             'dist_to_nonresidential_km','dist_to_residential_km']:
    minmax = gs.raster_info(name)
    print (name, minmax['min'], minmax['max'])

In [ ]:
gs.mapcalc("urban_change_01_11 = if(urban_2011 == 1, if(urban_2001 == 0, 1, null()), 0)")

Sample rasters

In [ ]:
gs.run_command('r.sample.category', input='urban_change_01_11', output='sampling',
               sampled='sa_counties,devpressure_30_05_01_2001,slope,road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_city_boundaries_km,dist_to_interchanges_km,dist_to_downtown_km,dist_to_planned_dev_km,dist_to_commercial_km,dist_to_industrial_km,dist_to_low_den_residential_km,dist_to_high_den_residential_km,dist_to_mixeduse_km,dist_to_nonresidential_km,dist_to_residential_km',
               npoints=[1000, 400])